As electronic vehicles (EVs) become more popular, there is an increasing need for access to charging stations, also known as ports. To that end, many modern apartment buildings have begun retrofitting their parking garages to include shared charging stations. A charging station is shared if it is accessible by anyone in the building.

<img src="charging_station.jpg" alt="EV Charging" width="500" height="auto">

But with increasing demand comes competition for these ports — nothing is more frustrating than coming home to find no charging stations available! In this project, you will use a dataset to help apartment building managers better understand their tenants’ EV charging habits.

The data has been loaded into an Azure Databricks database, containing a schema called `vehicles` and a single table named `charging_sessions` with the following columns:

## vehicles.charging_sessions

| Column | Definition | Data type |
|-|-|-|
|`garage_id`| Identifier for the garage/building|`STRING`|
|`user_id` | Identifier for the individual user|`STRING`|
|`user_type`|Indicating whether the station is `Shared` or `Private`| `STRING` |
|`start_plugin`|The date and time the session started |`TIMESTAMP`|
|`start_plugin_hour`|The hour (in military time) that the session started | `NUMERIC`|
|`end_plugout`|The date and time the session ended | `TIMESTAMP` |
|`end_plugout_hour`|The hour (in military time) that the session ended | `NUMERIC`|
|`duration_hours`| The length of the session, in hours|`NUMERIC`|
|`el_kwh`| Amount of electricity used (in Kilowatt hours)|`NUMERIC`|
|`month_plugin`| The month that the session started |`STRING`|
|`weekdays_plugin`| The day of the week that the session started|`STRING`|

Let’s get started!

#### Sources
- **Data**: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via [Kaggle](https://www.kaggle.com/datasets/anshtanwar/residential-ev-chargingfrom-apartment-buildings),
- **Image**: Julian Herzog, [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via Wikimedia Commons

In [47]:
-- inspecting_dataset

SELECT *
FROM VEHICLES.CHARGING_SESSIONS
LIMIT 10

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,AdO3,AdO3-4,Private,2018-12-21 10:20:00+00:00,10,2018-12-21 10:23:00+00:00,10,0.30,0.050000,Dec,Friday
1,AdO3,AdO3-4,Private,2018-12-21 10:24:00+00:00,10,2018-12-21 10:32:00+00:00,10,0.87,0.136667,Dec,Friday
2,AdO3,AdO3-4,Private,2018-12-21 11:33:00+00:00,11,2018-12-21 19:46:00+00:00,19,29.87,8.216389,Dec,Friday
3,AdO3,AdO3-2,Private,2018-12-22 16:15:00+00:00,16,2018-12-23 16:40:00+00:00,16,15.56,24.419722,Dec,Saturday
4,AdO3,AdO3-2,Private,2018-12-24 22:03:00+00:00,22,2018-12-24 23:02:00+00:00,23,3.62,0.970556,Dec,Monday
5,AdO3,AdO3-2,Private,2018-12-24 23:32:00+00:00,23,2018-12-25 17:37:00+00:00,17,16.14,18.077778,Dec,Monday
6,AdO3,AdO3-2,Private,2018-12-25 18:25:00+00:00,18,2018-12-26 16:08:00+00:00,16,10.33,21.720833,Dec,Tuesday
7,AdO3,AdO3-4,Private,2018-12-26 10:41:00+00:00,10,2018-12-26 16:52:00+00:00,16,27.66,6.188056,Dec,Wednesday
8,AdO3,AdO3-2,Private,2018-12-26 18:46:00+00:00,18,2018-12-26 21:08:00+00:00,21,8.83,2.371111,Dec,Wednesday
9,AdO3,AdO3-2,Private,2018-12-29 16:04:00+00:00,16,2018-12-29 20:55:00+00:00,20,8.58,4.856111,Dec,Saturday


In [43]:
-- unique_users_per_garage

SELECT
    garage_id,
    COUNT(DISTINCT user_id) AS num_unique_users
FROM
    VEHICLES.CHARGING_SESSIONS
WHERE
    user_type = 'Shared'
GROUP BY
    garage_id
ORDER BY
    num_unique_users DESC;

,garage_id,num_unique_users
0,Bl2,18
1,AsO2,17
2,UT9,16
3,AdO3,3
4,SR2,2
5,MS1,2
6,AdA1,1
7,Ris,1


In [44]:
-- most_popular_shared_start_times

SELECT
    weekdays_plugin,
    start_plugin_hour,
    COUNT(*) AS num_charging_sessions
FROM
    vehicles.charging_sessions
WHERE
    user_type = 'Shared'
GROUP BY
    weekdays_plugin,
    start_plugin_hour
ORDER BY
    num_charging_sessions DESC
LIMIT
    10;

,weekdays_plugin,start_plugin_hour,num_charging_sessions
0,Sunday,17,30
1,Friday,15,28
2,Thursday,16,26
3,Thursday,19,26
4,Sunday,15,25
5,Wednesday,19,25
6,Sunday,18,25
7,Friday,16,24
8,Monday,15,24
9,Saturday,17,23


In [48]:
-- long_duration_shared_users

SELECT
    user_id,
    AVG(duration_hours) as avg_charging_duration
FROM
    vehicles.charging_sessions
WHERE
    user_type = 'Shared'
GROUP BY
    user_id
HAVING
    AVG(duration_hours) > 10
ORDER BY
    AVG(duration_hours) DESC;

,user_id,avg_charging_duration
0,Share-9,16.845833
1,Share-17,12.894556
2,Share-25,12.214475
3,Share-18,12.088807
4,Share-8,11.550431
5,AdO3-1,10.369387
